# Lie Detection: Reliability of Relevant Predictors in Classifiers

[INTRODUCTION OF THE PROBLEM HERE]

### Datasets:
[ADD AND COMMENT ABOUT DATASETS HERE]
### Tasks:
[ADD TASK HERE]

### Integrants:
Name, Matricola, Email respectively
- Santiago Víquez Segura, 2048722, santiago.viquezsegura@studenti.unipd.it
- Phillippe Robert, 2041505, philippe.robert@studenti.unipd.it
- Huzaifa Fazal,

## Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.ensemble import  RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Load datasets

In [3]:
BF_CTU_df = pd.read_csv("data/BF_df_CTU.csv", delimiter=",")
BF_OU_df = pd.read_csv("data/BF_df_OU.csv", delimiter=",")
BF_V_df = pd.read_csv("data/BF_df_V.csv", delimiter=",")
DT_CC_df = pd.read_csv("data/DT_df_CC.csv", delimiter=",")
DT_JI_df = pd.read_csv("data/DT_df_JI.csv", delimiter=";")
IADQ_df = pd.read_csv("data/IADQ_df.csv", delimiter=",")
IESR_df = pd.read_csv("data/IESR_df.csv", delimiter=",")
NAQR_df = pd.read_csv("data/NAQ_R_df.csv", delimiter=";")
PCL5_df = pd.read_csv("data/PCL5_df.csv", delimiter=";")
PHQ9_df = pd.read_csv("data/PHQ9_GAD7_df.csv", delimiter=";")
PID5_df = pd.read_csv("data/PID5_df.csv", delimiter=";")
PRFQ_df = pd.read_csv("data/PRFQ_df.csv", delimiter=",")
PRMQ_df = pd.read_csv("data/PRMQ_df.csv", delimiter=",")
DDDT_df = pd.read_csv("data/RAW_DDDT.CSV", delimiter=",")
SPID5_df = pd.read_csv("data/sPID-5_df.csv", delimiter=",")
RNEOPI_honest_df = pd.read_excel("data/Honest.xlsx", header=1)
RNEOPI_fake_df = pd.read_excel("data/Faked.xlsx", header=1)

## Baseline Feature Importance

In [3]:
def evaluate_clf(df, clf, cv):
    """
    Performs cross-validation of a classifier over a dataset
    
    Args:
        df: (pd.DataFrame) train dataset
        clf: (sklearn fitted estimator)
        cv: (int) number of folds
        
    Returns:
        output: (dict) {fit_time:[...], score_time:[...], estimator:[...], test_score:[...]
    """
    
    X = df.loc[:, df.columns != 'CONDITION'] # select all columns except CONDITION
    y = df["CONDITION"]
    output = cross_validate(clf, X, y, cv=cv, scoring='accuracy', return_estimator=True)
    
    return output

In [19]:
def get_feature_importance(model_output):
    """
    Averages feature importances / coefficient values from all the fitted estimators
    
    Args:
        model_output: (list) sklearn fitted estimators
    
    Retruns: (dict) {predictor: value}
    """
    
    n_folds = len(output["estimator"])
    n_features = output["estimator"][0].n_features_in_

    coeficients = np.zeros(n_features)
    for estimator in output["estimator"]:
        if isinstance(estimator, LogisticRegression):
            coeficients += estimator.coef_[0] # TODO: RF uses feature_importances insted of coef_
        else:
            coeficients += estimator.feature_importances_[0]
    coeficients = coeficients/n_folds
    res = dict(zip(estimator.feature_names_in_, coeficients))

    return res

### Logistic Regression

In [20]:
clf = LogisticRegression(solver="lbfgs")
output = evaluate_clf(BF_CTU_df, clf, cv=5)

accuracy = np.mean(output["test_score"])
feature_importance = get_feature_importance(output)

print(f"Cross-Validated Accuracy: {accuracy}")
print(feature_importance)

Cross-Validated Accuracy: 0.8258937691521961
{'EX1G': -0.35843799604817717, 'EX2G': 0.11496166274546744, 'A1G': -0.6705075183900067, 'A2G': 0.4619940380891191, 'C1G': -0.8383693181328586, 'C2G': 1.0338735349617285, 'ES1G': -0.5942175600772364, 'ES2G': -0.6924994626296705, 'O1G': 0.1437837905444445, 'O2G': -0.13216065806241742}


### Random Forest

In [22]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
output = evaluate_clf(BF_CTU_df, clf, cv=5)

accuracy = np.mean(output["test_score"])
feature_importance = get_feature_importance(output)

print(f"Cross-Validated Accuracy: {accuracy}")
print(feature_importance)

Cross-Validated Accuracy: 0.8440245148110316
{'EX1G': 0.06760551682051989, 'EX2G': 0.06760551682051989, 'A1G': 0.06760551682051989, 'A2G': 0.06760551682051989, 'C1G': 0.06760551682051989, 'C2G': 0.06760551682051989, 'ES1G': 0.06760551682051989, 'ES2G': 0.06760551682051989, 'O1G': 0.06760551682051989, 'O2G': 0.06760551682051989}
